In [1]:
!pip install codeflare_sdk==0.16.3

  Attempting uninstall: codeflare_sdk
    Found existing installation: codeflare-sdk 0.16.1
    Uninstalling codeflare-sdk-0.16.1:
      Successfully uninstalled codeflare-sdk-0.16.1

[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [3]:
cluster_name = "cluster-m5t2c"
namespace = "test-llms"

In [4]:
cluster = Cluster(ClusterConfiguration(name=cluster_name, 
                                       namespace=namespace, # Update to your namespace
                                       num_workers=2, 
                                       min_cpus=4, 
                                       max_cpus=4, 
                                       min_memory=8, 
                                       max_memory=8, 
                                       head_gpus=0,
                                       num_gpus=0,
                                       image="quay.io/project-codeflare/ray:latest-py39-cu118",
                                       write_to_file=False, # When enabled Ray Cluster yaml files are written to /HOME/.codeflare/resources 
                                       local_queue="local-queue"
                                       ))

Yaml resources loaded for cluster-m5t2c


In [5]:
# Create the Ray cluster
cluster.up()

RuntimeError: Failed to get RayCluster CustomResourceDefinition: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Audit-Id': 'b109ca1d-5392-4803-8fb4-a0f9fdbd3646', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'X-Kubernetes-Pf-Flowschema-Uid': 'dde4b2ef-aab7-48db-9905-efb4f234bf5f', 'X-Kubernetes-Pf-Prioritylevel-Uid': '0c4052e4-2163-4ba3-b370-0e68e74048cf', 'Date': 'Wed, 03 Jul 2024 15:59:08 GMT', 'Content-Length': '345'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"rayclusters.ray.io is forbidden: User \"system:serviceaccount:test-llms:fine-tune-llama\" cannot list resource \"rayclusters\" in API group \"ray.io\" in the namespace \"test-llms\"","reason":"Forbidden","details":{"group":"ray.io","kind":"rayclusters"},"code":403}



In [15]:
cluster.wait_ready()

Waiting for requested resources to be set up...


KeyboardInterrupt: 

In [ ]:
cluster.details()

In [40]:
# Initialize the Job Submission Client
client = cluster.job_client

In [ ]:
# The S3 bucket where to store checkpoint.
# It can be set manually, otherwise it's retrieved from configured the data connection.
s3_bucket = ""
if not s3_bucket:
    s3_bucket = os.environ.get('AWS_S3_BUCKET')
assert s3_bucket, "An S3 bucket must be provided to store checkpoints"

In [ ]:
submission_id = client.submit_job(
    entrypoint="python ray_finetune_llm_deepspeed.py "
               "--model-name=TinyLlama/TinyLlama_v1.1 "
               "--lora "
               "--num-devices=8 "
               "--num-epochs=3 "
               "--ds-config=./deepspeed_configs/zero_3_llama_2_7b.json "
               f"--storage-path=s3://{s3_bucket}/ray_finetune_llm_deepspeed/ "
               "--batch-size-per-device=32 "
               "--eval-batch-size-per-device=32 ",
    runtime_env={
        "env_vars": {
            "AWS_ACCESS_KEY_ID": os.environ.get('AWS_ACCESS_KEY_ID'),
            "AWS_SECRET_ACCESS_KEY": os.environ.get('AWS_SECRET_ACCESS_KEY'),
            "AWS_DEFAULT_REGION": os.environ.get('AWS_DEFAULT_REGION')
        },
        "pip": "requirements.txt",
        "working_dir": "./",
        "excludes": ["/docs/", "*.ipynb", "*.md"]
    },
)
print(submission_id)

In [ ]:
client.stop_job(submission_id)

In [42]:
cluster.down()